In [ ]:
# Загрузим данные с помощью модуля Pandas в объект DataFrame
import pandas as pd
omega_stability = pd.read_csv('omega_stability.csv')

# Проведем первичную обработку данных
omega_stability = omega_stability.T
omega_stability = omega_stability.drop(index = "Name")
omega_stability.columns = ["Omega stability", "Survival time"]

# Преобразуем значения ячеек в числовые
omega_stability["Omega stability"] = omega_stability["Omega stability"].astype(float)
omega_stability["Survival time"] = omega_stability["Survival time"].astype(float)

x = omega_stability["Omega stability"].values
y = omega_stability["Survival time"].values

# Проведем визуализацию
omega_stability.plot(kind="scatter", x="Omega stability", y="Survival time")

In [ ]:
# Построение LTS-оценки
def c_step(x, y, x0, model_func, tolerance, max_iter, h0):
    import math
    import random
    import numpy as np
    from scipy.optimize import least_squares
    from scipy.optimize import curve_fit 
    residual_func = lambda a, x, y: model_func(x, a) - y
    i = 1
    H1 = random.sample(range(len(x)), h0)
    Q1 = 9999999
    prev = 0
    error = abs(Q1-prev)
    theta = []
    # Получение x0
    x0 = least_squares(residual_func, x0, loss="linear", args=(x, y)).x
    while error > tolerance and i < max_iter:
        theta = least_squares(residual_func, x0, loss="linear", args=(x[H1], y[H1])).x
        est = model_func(x, theta)
        e = y - est
        prev = Q1
        Q1 = sum(e**2)
        error = abs(Q1 - prev)
        pi = np.argsort(np.abs(e))
        H1 = pi[:h0]
        i += 1
        #print(error)
    return theta

In [ ]:
def loo_cross_validation_lts(x, y, model_func, h0):
    import math
    import numpy as np
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import LeaveOneOut
    loo = LeaveOneOut()
    predicts = []    
    for train, test in loo.split(x):
        x_train = x[train]
        y_train = y[train]
        x_test = x[test]
        y_test = y[test]
        a = c_step(x, y, [0, 0, 0, 0], model_func, 1e-7, 500, h0)
        predict = model_func(x_test, a)
        predicts.append(predict)
    rmse = math.sqrt(mean_squared_error(y, predicts))
    return rmse, a

In [ ]:
def cube_func(x, a):
    return a[0] + a[1] * x + a[2] * x**2 + a[3] * x**3
def cube_func_ols(x, a0, a1, a2, a3):
    return a0 + a1 * x + a2 * x**2 + a3 * x**3

In [20]:
d = {}
cv_m, a_m = 300, []
h0 = 0
for i in range(22, 40):
    cv, a = loo_cross_validation_lts(x, y, cube_func, i)
    d[i] = a
    if cv < cv_m:
        cv_m = cv
        a_m = a
        h0 = i
    print("i " + str(i))
    print(cv)
print("min cv ", cv_m)
print("h0", h0)
print(a_m)

i 22
5.420191192809716
i 23
9.84152174127635
i 24
6.995632620424497
i 25
3.4137603642886636
i 26
3.6382498465488853
i 27
4.6543846453215165
i 28
3.568676923030016
i 29
3.607326079834195
i 30
3.6375599791058293
i 31
3.802599810514567
i 32
3.5726083894093943
i 33
3.727328034683732
i 34
3.690986347381327
i 35
3.5736170245236325
i 36
3.5625996046079997
i 37
3.550019357946926
i 38
3.470243495076802
i 39
3.444861579883074
min cv  3.4137603642886636
h0 25
[ 15.95804684 124.23325072 353.48388773 325.50331668]


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots()
ax.scatter(x, y)
x_plot = np.linspace(-0.5, 0.5, num=1000)
# ols
from scipy.optimize import curve_fit
a, _ = curve_fit(cube_func_ols, x, y)
print(a)
y_plot = cube_func(x_plot, a)
ax.plot(x_plot, y_plot, c="r")
#ax.legend(("ols", "data"))

#37
#h0 = 37
y_plot = cube_func(x_plot, d[h0])
ax.plot(x_plot, y_plot, c="g")
ax.legend(("ols", h0, "data"))

In [ ]:
# M-estimators

In [ ]:
from scipy.optimize import least_squares
residual_func = lambda a, x, y: cube_func(x, a) - y
theta = least_squares(residual_func, [13.2172348, 59.92121269, -14.74252034, -249.17161382], loss="huber", args=(x, y))
a_m = theta.x
fig, ax = plt.subplots()
ax.scatter(x, y)
x_plot = np.linspace(-0.5, 0.5, num=1000)
y_plot = a_m[0] + a_m[1]*x_plot + a_m[2]*x_plot**2 + a_m[3]*x_plot**3
ax.plot(x_plot, y_plot)

In [ ]:
def loo_cross_validation_robust(x, y, model_func):
    import math
    from scipy.optimize import curve_fit 
    import numpy as np
    from sklearn.metrics import mean_squared_error
    from sklearn.model_selection import LeaveOneOut
    residual_func = lambda a, x, y: model_func(x, a) - y
    loo = LeaveOneOut()
    predicts = []    
    for train, test in loo.split(x):
        x_train = x[train]
        y_train = y[train]
        x_test = x[test]
        y_test = y[test]
        robust = least_squares(residual_func, [13.2172348, 59.92121269, -14.74252034, -249.17161382], f_scale = 0.1, loss='arctan', args=(x_train, y_train))
        param = robust.x
        predict = model_func(x_test, param)
        predicts.append(predict)
    rmse = math.sqrt(mean_squared_error(y, predicts))
    return rmse

In [ ]:
print(loo_cross_validation_robust(x, y, cube_func))

In [ ]:
#----------------------------------------------------------------------------------------------------

In [ ]:
def lin_ols(x, a0, a1):
    return a0 + a1*x;

In [ ]:
# Новая LOO кросс-валидация
def new_loo_cv(x, y, model_func):
    import math
    from scipy.optimize import curve_fit 
    import numpy as np
    SE = 0
    for i in range(len(x)):
        test_x = x[i]
        test_y = y[i]
        x_new = np.delete(x, i)
        y_new = np.delete(y, i)
        a, _ = curve_fit(model_func, x_new, y_new)
        exact_val = model_func(test_x, a[0], a[1])
        SE+=(test_y - exact_val)**2
    RMSE = math.sqrt(SE/len(x))
    return RMSE

In [ ]:
print(new_loo_cv(x, y, cube_func_ols))

In [ ]:
print(new_loo_cv(x, y, lin_ols))

In [ ]:
# Новая LOO кросс-валидация для LTS
def new_loo_cv_lts(x, y, model_func, h0):
    import math
    from scipy.optimize import curve_fit 
    import numpy as np
    SE = 0
    for i in range(len(x)):
        test_x = x[i]
        test_y = y[i]
        x_new = np.delete(x, i)
        y_new = np.delete(y, i)
        a = c_step(x_new, y_new, [0, 0, 0, 0], model_func, 1e-7, 500, h0)
        exact_val = model_func(test_x, a)
        SE+=(test_y - exact_val)**2
    RMSE = math.sqrt(SE/len(x))
    return (RMSE, a)

In [ ]:
for h in range(20, 39):
    print(h)
    print(new_loo_cv_lts(x, y, cube_func, h))

In [ ]:
# Новая LOO кросс-валидация для LTS
def new_loo_cv_robust(x, y, model_func):
    #from IPython.core.debugger import set_trace
    import math
    from scipy.optimize import curve_fit 
    import numpy as np
    from scipy.optimize import least_squares
    SE = 0
    residual_func = lambda a, x, y: model_func(x, a) - y
    for i in range(len(x)):
        test_x = x[i]
        test_y = y[i]
        x_new = np.delete(x, i)
        y_new = np.delete(y, i)
        #set_trace()
        ak = least_squares(residual_func, [ 12.90005922,  73.80893852, 122.45713168,  19.40206171], 
                           loss='huber', args=(x_new, y_new)).x

        exact_val = model_func(test_x, ak)
        SE+=(test_y - exact_val)**2
    RMSE = math.sqrt(SE/len(x))
    return (RMSE, ak)

In [ ]:
print(new_loo_cv_robust(x, y, cube_func))

In [ ]:
from scipy.optimize import least_squares
#residual_func = lambda a, x, y: cube_func(x, a) - y
#theta = least_squares(residual_func, [13.2172348, 59.92121269, -14.74252034, -249.17161382], loss="huber", args=(x, y))
a_m = [  12.42743864,   55.43049293,   -5.49440223, -219.91135959]
fig, ax = plt.subplots()
ax.scatter(x, y)
x_plot = np.linspace(-0.5, 0.5, num=1000)
y_plot = a_m[0] + a_m[1]*x_plot + a_m[2]*x_plot**2 + a_m[3]*x_plot**3
ax.plot(x_plot, y_plot)

a_m = [ 11.41492391,   50.88183143,    0.8377467 , -177.42188028]
x_plot = np.linspace(-0.5, 0.5, num=1000)
y_plot = a_m[0] + a_m[1]*x_plot + a_m[2]*x_plot**2 + a_m[3]*x_plot**3
ax.plot(x_plot, y_plot)
# оранжевый - LTS

M-оценка не дает минимального результата, т.к. LTS на примерно таких же коэффициентах выдаёт ещё большее значение Кросс-Валидации

In [ ]:
x_new = x + abs(np.min(x))

In [ ]:
fig, ax = plt.subplots()
x_plot = np.linspace(0, 1, num=1000)
y_plot = weibull([0.5, 5], x_plot)
ax.plot(x_plot, y_plot)

In [ ]:
# теперь подготовим исходные данные 
# выполним сортировку
x, y = zip(*sorted(zip(x_new, y)))
x = np.asarray(x)
y = np.asarray(y)

In [ ]:
# визуадизируем новые данные
plt.scatter(x, y)


In [ ]:
plt.plot(x, y)

In [ ]:
def weibull(a, x):
    return 21.3*(1-np.exp(-(x/a[0])**a[1]))

In [ ]:
# определим функцию отклонений
def weibull_residual(a, x, y):
    return 21.3*(1-np.exp(-(x/a[0])**a[1])) - y

In [ ]:
res_robust = least_squares(weibull_residual, [1, 1], loss='huber', args=(x, y)).x

In [ ]:
res_robust

In [ ]:
fig, ax = plt.subplots()
x_plot = x
y_plot = weibull(res_robust, x_plot)
ax.plot(x_plot, y_plot)
ax.scatter(x, y)

In [ ]:
def new_loo_cv_robust(x, y, model_func):
    import math
    import numpy as np
    from scipy.optimize import least_squares
    from IPython.core.debugger import set_trace
    SE = 0
    residual_func = lambda a, x, y: model_func(a, x) - y
    for i in range(len(x)):
        x_test = x[i]
        y_test = y[i]
        x_train = np.delete(x, i)
        y_train = np.delete(y, i)
        #set_trace()
        an = least_squares(residual_func, np.ones(2), loss='soft_l1', args=(x_train, y_train)).x
        exact_val = model_func(an, x_test)
        SE+=(y_test - exact_val)**2
    RMSE = math.sqrt(SE/len(x))
    return (RMSE, an)

In [ ]:
new_loo_cv_robust(x, y, weibull)

In [ ]:
plt.scatter(x, y)
x_plot = x
y_plot = weibull([0.41788069, 3.45097678], x_plot)
plt.plot(x_plot, y_plot)